# Import thư viện cần thiết

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

# Import Random Forest Classifier model library
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.impute import SimpleImputer

from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

Đọc dữ liệu

In [2]:
train = pd.read_csv('../data/train_preprocessed_DK.csv')
train.head()

,Id,v2a1,hacdor,rooms,hacapo,v14a,refrig,v18q,v18q1,r4h1,...,SQBage,SQBhogar_total,SQBedjefe,SQBhogar_nin,SQBovercrowding,SQBdependency,SQBmeaned,agesq,Target,rez_esc-missing
0,ID_279628684,190000.0,0,3,0,1,1,0,0.0,0,...,1849,1,100,0,1.000000,0.0,100.0,1849,4,False
1,ID_f29eb3ddd,135000.0,0,4,0,1,1,1,1.0,0,...,4489,1,144,0,1.000000,64.0,144.0,4489,4,False
2,ID_68de51c94,0.0,0,8,0,1,1,0,0.0,0,...,8464,1,0,0,0.250000,64.0,121.0,8464,4,False
3,ID_d671db89c,180000.0,0,5,0,1,1,1,1.0,0,...,289,16,121,4,1.777778,1.0,121.0,289,4,False
4,ID_d56d6f5f5,180000.0,0,5,0,1,1,1,1.0,0,...,1369,16,121,4,1.777778,1.0,121.0,1369,4,False


In [3]:
test = pd.read_csv('../data/test_preprocessed_DK.csv')
test.head()

,Id,v2a1,hacdor,rooms,hacapo,v14a,refrig,v18q,v18q1,r4h1,...,SQBescolari,SQBage,SQBhogar_total,SQBedjefe,SQBhogar_nin,SQBovercrowding,SQBdependency,SQBmeaned,agesq,rez_esc-missing
0,ID_2f6873615,0.0,0,5,0,1,1,0,0.0,1,...,0,16,9,0,1,2.25,0.25,272.25,16,False
1,ID_1c78846d2,0.0,0,5,0,1,1,0,0.0,1,...,256,1681,9,0,1,2.25,0.25,272.25,1681,False
2,ID_e5442cf6a,0.0,0,5,0,1,1,0,0.0,1,...,289,1681,9,0,1,2.25,0.25,272.25,1681,False
3,ID_a8db26a79,0.0,0,14,0,1,1,1,1.0,0,...,256,3481,1,256,0,1.00,0.00,256.00,3481,False
4,ID_a62966799,175000.0,0,4,0,1,1,1,1.0,0,...,121,324,1,0,1,0.25,64.00,NaN,324,False


In [4]:
ntrain = train.shape[0]
ntest = test.shape[0]

all_data = pd.concat((train, test)).reset_index(drop=True)
all_data.tail()

,Id,v2a1,hacdor,rooms,hacapo,v14a,refrig,v18q,v18q1,r4h1,...,SQBage,SQBhogar_total,SQBedjefe,SQBhogar_nin,SQBovercrowding,SQBdependency,SQBmeaned,agesq,Target,rez_esc-missing
29989,ID_a065a7cad,0.0,1,2,1,1,1,0,0.0,0,...,100,36,25,4,36.0,0.25,33.0625,100,NaN,False
29990,ID_1a7c6953b,0.0,0,3,0,1,1,0,0.0,0,...,2916,16,36,4,4.0,1.00,36.0000,2916,NaN,False
29991,ID_07dbb4be2,0.0,0,3,0,1,1,0,0.0,0,...,144,16,36,4,4.0,1.00,36.0000,144,NaN,False
29992,ID_34d2ed046,0.0,0,3,0,1,1,0,0.0,0,...,144,16,36,4,4.0,1.00,36.0000,144,NaN,False
29993,ID_34754556f,0.0,0,3,0,1,1,0,0.0,0,...,2601,16,36,4,4.0,1.00,36.0000,2601,NaN,False


# Quan sát và xử lý dữ liệu đầu vào

## Quan sát dữ liệu

Kích thước dữ liệu.

In [5]:
print('Kích thước tập dữ liệu', all_data.shape, '. Trong đó:\n',
      '- Tập dữ liệu huấn luyện:', train.shape, ';\n',
      '- Tập dữ liệu kiểm thử:', test.shape, '.')

Kích thước tập dữ liệu (29994, 144) . Trong đó:
 - Tập dữ liệu huấn luyện: (8608, 144) ;
 - Tập dữ liệu kiểm thử: (21386, 143) .


Sự chênh lệch dữ liệu là do tập dữ liệu test không chứa biến dự đoán `target`.

Quan sát các biến định tính và định lượng.

In [6]:
numerical_cols = all_data.select_dtypes(include=np.number).columns.tolist()
categorical_cols = list(set(all_data.columns).difference(set(numerical_cols)))

categorical_cols

['Id', 'idhogar', 'rez_esc-missing']

Đây là bài toán phân lớp, nên ta cần xác định lại kiểu dữ liệu của biến dự đoán.

In [7]:
all_data['Target'] = all_data['Target'].astype('category')

In [8]:
numerical_cols = all_data.select_dtypes(include=np.number).columns.tolist()
categorical_cols = list(set(all_data.columns).difference(set(numerical_cols)))

categorical_cols

['Id', 'Target', 'idhogar', 'rez_esc-missing']

## Xử lý dữ liệu

### Loại bỏ các thuộc tính không cần thiết

- Loại bỏ các biến `id_`.

In [9]:
all_data.drop(['Id', 'idhogar'], axis=1, inplace=True)
all_data

,v2a1,hacdor,rooms,hacapo,v14a,refrig,v18q,v18q1,r4h1,r4h2,...,SQBage,SQBhogar_total,SQBedjefe,SQBhogar_nin,SQBovercrowding,SQBdependency,SQBmeaned,agesq,Target,rez_esc-missing
0,190000.0,0,3,0,1,1,0,0.0,0,1,...,1849,1,100,0,1.000000,0.00,100.0000,1849,4.0,False
1,135000.0,0,4,0,1,1,1,1.0,0,1,...,4489,1,144,0,1.000000,64.00,144.0000,4489,4.0,False
2,0.0,0,8,0,1,1,0,0.0,0,0,...,8464,1,0,0,0.250000,64.00,121.0000,8464,4.0,False
3,180000.0,0,5,0,1,1,1,1.0,0,2,...,289,16,121,4,1.777778,1.00,121.0000,289,4.0,False
4,180000.0,0,5,0,1,1,1,1.0,0,2,...,1369,16,121,4,1.777778,1.00,121.0000,1369,4.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29989,0.0,1,2,1,1,1,0,0.0,0,2,...,100,36,25,4,36.000000,0.25,33.0625,100,NaN,False
29990,0.0,0,3,0,1,1,0,0.0,0,1,...,2916,16,36,4,4.000000,1.00,36.0000,2916,NaN,False
29991,0.0,0,3,0,1,1,0,0.0,0,1,...,144,16,36,4,4.000000,1.00,36.0000,144,NaN,False
29992,0.0,0,3,0,1,1,0,0.0,0,1,...,144,16,36,4,4.000000,1.00,36.0000,144,NaN,False


- Loại bỏ các biến trùng lặp ý nghĩa.

In [10]:
# Check columns that is duplicated
# duplicated_cols = [col for col in all_data.columns if all_data[col].duplicated().any()]
def getDuplicateColumns(df):
 
    # Create an empty set
    duplicateColumnNames = set()
 
    # Iterate through all the columns of dataframe
    for x in range(df.shape[1]):
 
        # Take column at xth index.
        col = df.iloc[:, x]
 
        # Iterate through all the columns
        for y in range(x + 1, df.shape[1]):
 
            # Take column at yth index.
            otherCol = df.iloc[:, y]
 
            # Check if two columns at x & y
            if col.equals(otherCol):
                duplicateColumnNames.add(df.columns.values[y])
 
    return list(duplicateColumnNames)

getDuplicateColumns(all_data)

['agesq', 'hhsize', 'tipovivi5', 'hogar_total']

In [11]:
all_data['rent_per_adult'] = all_data['v2a1']/all_data['hogar_adul']
all_data['rent_per_person'] = all_data['v2a1']/all_data['hhsize']

all_data['overcrowding_room_and_bedroom'] = (all_data['hacdor'] + all_data['hacapo'])/2

all_data['no_appliances'] = all_data['refrig'] + all_data['computer'] + all_data['television']

all_data['r4h1_percent_in_male'] = all_data['r4h1'] / all_data['r4h3']
all_data['r4m1_percent_in_female'] = all_data['r4m1'] / all_data['r4m3']
all_data['r4h1_percent_in_total'] = all_data['r4h1'] / all_data['hhsize']
all_data['r4m1_percent_in_total'] = all_data['r4m1'] / all_data['hhsize']
all_data['r4t1_percent_in_total'] = all_data['r4t1'] / all_data['hhsize']

all_data['adult'] = all_data['hogar_adul'] - all_data['hogar_mayor']
all_data['dependency_count'] = all_data['hogar_nin'] + all_data['hogar_mayor']
all_data['dependency'] = all_data['dependency_count'] / all_data['adult']
all_data['child_percent'] = all_data['hogar_nin']/all_data['hogar_total']
all_data['elder_percent'] = all_data['hogar_mayor']/all_data['hogar_total']
all_data['adult_percent'] = all_data['hogar_adul']/all_data['hogar_total']

all_data['rent_per_bedroom'] = all_data['v2a1']/all_data['bedrooms']
all_data['adults_per_bedroom'] = all_data['adult']/all_data['bedrooms']
all_data['child_per_bedroom'] = all_data['hogar_nin']/all_data['bedrooms']
all_data['male_per_bedroom'] = all_data['r4h3']/all_data['bedrooms']

all_data['female_per_bedroom'] = all_data['r4m3']/all_data['bedrooms']
all_data['bedrooms_per_person_household'] = all_data['hhsize']/all_data['bedrooms']

all_data['tablet_per_person_household'] = all_data['v18q1']/all_data['hhsize']
all_data['phone_per_person_household'] = all_data['qmobilephone']/all_data['hhsize']

all_data['age_12_19'] = all_data['hogar_nin'] - all_data['r4t1']

all_data['rent_per_room'] = all_data['v2a1']/all_data['rooms']
all_data['bedroom_per_room'] = all_data['bedrooms']/all_data['rooms']
all_data['elder_per_room'] = all_data['hogar_mayor']/all_data['rooms']
all_data['adults_per_room'] = all_data['adult']/all_data['rooms']
all_data['child_per_room'] = all_data['hogar_nin']/all_data['rooms']
all_data['male_per_room'] = all_data['r4h3']/all_data['rooms']
all_data['female_per_room'] = all_data['r4m3']/all_data['rooms']
all_data['room_per_person_household'] = all_data['hhsize']/all_data['rooms']
all_data['escolari_age'] = all_data['escolari']/all_data['age']

all_data['rez_esc_escolari'] = all_data['rez_esc']/all_data['escolari']
all_data['rez_esc_r4t1'] = all_data['rez_esc']/all_data['r4t1']
all_data['rez_esc_r4t2'] = all_data['rez_esc']/all_data['r4t2']
all_data['rez_esc_r4t3'] = all_data['rez_esc']/all_data['r4t3']
all_data['rez_esc_age'] = all_data['rez_esc']/all_data['age']

# Xây dựng mô hình học máy